In [2]:
from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing import image
import numpy as np
K.set_learning_phase(0)

#加载inceptionV3网络
model = inception_v3.InceptionV3(weights='imagenet', include_top = True)
#将每层网络的名称打印出来，其中带有"activation"开头的网络层就是留有”残影“的卷积层
for layer in model.layers:
  print(layer.name)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
act

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras.models import Model
#把带有"activation”的网络层全部抽取出来
activation_layers = [layer.output for layer in model.layers if layer.name.startswith("activation_")]
activation_model = Model(model.input, outputs = activation_layers)

In [0]:
from keras.preprocessing import image

#增加图片预处理代码，并把要使用的图片加载进来
def  preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  #将二维图片扩展到三维，最高维表示图片的数量，由于只有一张图片，因此最高维的数量设置为1
  img = np.expand_dims(img, axis = 0)
  img = inception_v3.preprocess_input(img)
  return img



In [0]:
layer_name = 'activation_41'
#执行公式(1)获取给定卷积层的输出
activation = model.get_layer(layer_name).output

'''
卷积层的输出是个多维向量，我们如何让一个多维向量增大呢？一个办法是求多维向量里面每个分量的平方和，
然后我们调整图片像素使得该平方和增大
'''
scaling = K.prod(K.cast(K.shape(activation), 'float32'))
'''
计算时我们要忽略图片边缘点,下面代码中第一个2:-2中，2表示忽略左边缘前2个像素点，-2表示忽略右边缘最后2个像素点。
第二个2：-2中，2表示忽略顶部头2个像素点，-2表示忽略忽略底部倒数2个像素点
'''
#loss对应函数（1）
loss = K.sum(K.square(activation[:, 2:-1, 2:-2, :])) / scaling

#对像素点求偏导数，对应函数（2）
dream = model.input
grads = K.gradients(loss, dream)[0]
#将偏导数转换为[0,1]之间的值
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

iterate_grad_ac_step = K.function([dream], [loss, grads])

#使用梯度上升发调整输入图片像素点
def  gradient_ascent(x, iterations, step, max_loss = None):
  for i in range(iterations):
    loss_value, grad_values = iterate_grad_ac_step([x])
    print('Loss value is at ', i, ':', loss_value)
    '''
    限制增大上限，如果卷积层输出过大，那有可能是输入图片像素点调整过头，这会
    导致图片视觉效果不好看
    '''
    if max_loss is not None and loss_value > max_loss:
      break
    x += step * grad_values
  return x

In [0]:
import scipy

def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    '''
    x格式为(1, 3, width, height), 其中3表示像素点RGB的值，我们需要把前面多余的1去掉，
    然后把格式转换为(width, height, 3)
    '''
    #去掉最高维度多余的1
    x = x.reshape((3, x.shape[2], x.shape[3]))
    #转换为（width, height, RGB)
    x = x.transpose((1, 2, 0))
  else:
    '''
    如果x的格式为(1, width, height, 3),那么我们去掉最高维度多余的1即可
    '''
    x = x.reshape((x.shape[1], x.shape[2], 3))
    
  '''
  图片在输入inceptionV3进行处理时，它会对图片像素点做如下变换：
  x /= 255.
  x -= 0.5
  x *= 2.
  因此我们要对它的输出做对应的反变换才能正常显示图片
  '''
  x /= 2.
  x += .5
  x *= 255.
  
  x = np.clip(x, 0, 255).astype('uint8')
  return x

def  resize_img(img, size):
  #将图片根据给定比例进行缩放
  img = np.copy(img)
  factors = (1, float(size[0]) / img.shape[1],
                float(size[1]) / img.shape[2],
             1
            )
  return scipy.ndimage.zoom(img, factors, order = 1)

def  save_img(img, fname):
  img = deprocess_image(np.copy(img))
  scipy.misc.imsave('/content/gdrive/My Drive/' + fname, img)
    

In [7]:
#将图片缩小4次
num_octave = 4
#缩小比例为1.4
octave_scale = 1.4

image_path = '/content/gdrive/My Drive/blue_sky.jpg'
img = preprocess_image(image_path)
print(img.shape)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
#将图片以比例1.4连续缩小
for i in range(1, num_octave):
  shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)
  
                        
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
#将图片缩小到最小形态
shrunk_original_img = resize_img(img, successive_shapes[0])
print(successive_shapes)

(1, 386, 580, 3)
[(140, 211), (196, 295), (275, 414), (386, 580)]


In [8]:
MAX_ITERATION = 20
MAX_LOSS = 20
learning_rate = 0.01

for shape in successive_shapes:
  print('Processing image shape: ', shape)
  #将上一步梯度上升法调整后的图片以1.4比例放大
  img = resize_img(img, shape)
 
  '''
  将图片缩小1.4倍，然后再放大1.4倍，一来一回会造成像素点信息损失，下面三行代码将损失记录下来
  '''
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
  same_size_original = resize_img(original_img, shape)
  lost_detail = same_size_original - upscaled_shrunk_original_img
  #下面代码相当于执行图12-60中加号对应操作
  img += lost_detail
  
   #使用梯度上升法取得每个像素点的调整幅度
  img = gradient_ascent(img, iterations = MAX_ITERATION,
                       step = learning_rate,
                       max_loss = MAX_LOSS)
  
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
  
save_img(img, fname='final_dream.png')

Processing image shape:  (140, 211)
Loss value is at  0 : 0.17687777
Loss value is at  1 : 0.3071308
Loss value is at  2 : 0.4573659
Loss value is at  3 : 0.6172608
Loss value is at  4 : 0.78259045
Loss value is at  5 : 0.9258283
Loss value is at  6 : 1.0947613
Loss value is at  7 : 1.2477114
Loss value is at  8 : 1.4072728
Loss value is at  9 : 1.5584034
Loss value is at  10 : 1.7039208
Loss value is at  11 : 1.853727
Loss value is at  12 : 1.9702474
Loss value is at  13 : 2.0778344
Loss value is at  14 : 2.1962552
Loss value is at  15 : 2.3339348
Loss value is at  16 : 2.4356647
Loss value is at  17 : 2.5728376
Loss value is at  18 : 2.6820874
Loss value is at  19 : 2.7637002
Processing image shape:  (196, 295)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Loss value is at  0 : 0.4048001
Loss value is at  1 : 0.71947783
Loss value is at  2 : 0.9570125
Loss value is at  3 : 1.1783383
Loss value is at  4 : 1.3832954
Loss value is at  5 : 1.5628526
Loss value is at  6 : 1.7058357
Loss value is at  7 : 1.8806349
Loss value is at  8 : 2.0349624
Loss value is at  9 : 2.1661477
Loss value is at  10 : 2.3290172
Loss value is at  11 : 2.4676006
Loss value is at  12 : 2.5842729
Loss value is at  13 : 2.695279
Loss value is at  14 : 2.841451
Loss value is at  15 : 2.9296634
Loss value is at  16 : 3.0645425
Loss value is at  17 : 3.1719193
Loss value is at  18 : 3.285319
Loss value is at  19 : 3.3802764
Processing image shape:  (275, 414)
Loss value is at  0 : 0.46704948
Loss value is at  1 : 0.80155885
Loss value is at  2 : 1.0665408
Loss value is at  3 : 1.2956998
Loss value is at  4 : 1.4996808
Loss value is at  5 : 1.7121378
Loss value is at  6 : 1.8910944
Loss value is at  7 : 2.0419438
Loss value is at  8 : 2.2079356
Loss value is at  9 : 2.36